In [1]:
library(data.table)
library(conveniencefunctions)
library(VennDiagram)

Loading required package: ggplot2



|Function                      |Shortcut         |
|:-----------------------------|:----------------|
|alignAssign                   |Ctrl+Q           |
|alignCursor                   |Ctrl+Shift+Q     |
|toggle_subsection             |Ctrl+Y           |
|toggle_subsubsection          |Ctrl+Shift+Y     |
|insertFormals                 |Ctrl+Shift+Alt+I |
|assignFormals                 |Ctrl+Shift+Alt+O |
|fixCommas                     |Ctrl+Alt+Z       |
|exposeAsArgument              |Ctrl+Alt+C       |
|toggle_mclapply               |Ctrl+Shift+G     |
|insert_debugonce              |Ctrl+Shift+Alt+G |
|transform_subsection          |Ctrl+J           |
|initiate_or_delete_subsection |Ctrl+Shift+J     |
|renumber_sections             |Ctrl+Shift+R     |
|insert_loopdebugger           |Ctrl+Shift+I     |
|toggle_blabla                 |Ctrl+Alt+O       |
|extract_importFrom            |Ctrl+Alt+I       |
|refactor_functionCall         |Ctrl+Alt+U       |
|insertHistory                 

Loading required package: grid

Loading required package: futile.logger



# Load data and check for exceptions

In [2]:
dat <- fread('McPAS-TCR.csv', data.table = F)
dim(dat)
head(dat)

[1] 39664    29

,CDR3.alpha.aa,CDR3.beta.aa,Species,Category,Pathology,Pathology.Mesh.ID,Additional.study.details,Antigen.identification.method,Single.cell,NGS,⋯,TRAV,TRAJ,TRBV,TRBD,TRBJ,Reconstructed.J.annotation,CDR3.beta.nt,Mouse.strain,PubMed.ID,Remarks
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>
1,NA,CASSDAGANTEVF,Mouse,Pathogens,Lymphocytic choriomeningitis virus (LCMV),D008217,NA,2.1,No,No,⋯,NA,NA,TRBV8-1,NA,TRBJ1-1,No,NA,P14,1716213,NA
2,NA,CASSDAGAYAEQF,Mouse,Pathogens,Lymphocytic choriomeningitis virus (LCMV),D008217,NA,2.1,No,No,⋯,NA,NA,TRBV8-1,NA,TRBJ2-1,No,NA,P14,1716213,NA
3,NA,CASSDAGGAAEVF,Mouse,Pathogens,Lymphocytic choriomeningitis virus (LCMV),D008217,NA,2.1,No,No,⋯,NA,NA,TRBV8-3,NA,TRBJ1-1,No,NA,P14,1716213,NA
4,NA,CASSDAGHSPLYF,Mouse,Pathogens,Lymphocytic choriomeningitis virus (LCMV),D008217,NA,2.1,No,No,⋯,NA,NA,TRBV8-1,NA,TRBJ1-6,No,NA,P14,1716213,NA
5,NA,CASSDAWGGAEQYF,Mouse,Pathogens,Lymphocytic choriomeningitis virus (LCMV),D008217,NA,2.1,No,No,⋯,NA,NA,TRBV8-3,NA,TRBJ2-6,No,NA,P14,1716213,NA
6,NA,CASSDGANTEVF,Mouse,Pathogens,Lymphocytic choriomeningitis virus (LCMV),D008217,NA,2.1,No,No,⋯,NA,NA,TRBV8-1,NA,TRBJ1-1,No,NA,P14,1716213,NA


In [3]:
dat$CDR3.beta.aa <- gsub('^C', '', dat$CDR3.beta.aa)
dat$CDR3.beta.aa <- gsub('F$', '', dat$CDR3.beta.aa)

In [4]:
any(is.na(dat))

[1] TRUE

In [5]:
# check for special characters in CDR3 and epitope sequences
any(grepl("[[:punct:]]", dat$CDR3.beta.aa))
any(grepl("[[:punct:]]", dat$Epitope.peptide))

[1] TRUE

[1] TRUE

In [6]:
# check for special characters in CDR3 and epitope sequences
any(grepl(" ", dat$CDR3.beta.aa))
any(grepl(" ", dat$Epitope.peptide))

[1] FALSE

[1] FALSE

In [7]:
# remove unclear CDR3
dat <- dat[!is.na(dat$CDR3.beta.aa) & !grepl("[[:punct:]]", dat$CDR3.beta.aa),]
dat <- dat[!grepl('[a-z]', dat$CDR3.beta.aa), ]
dim(dat)

[1] 37132    29

In [8]:
unique(dat$Species)

[1] "Mouse" "Human" NA

In [9]:
dat <- dat[!is.na(dat$Species) & dat$Species != 'Mouse', 
           c('CDR3.beta.aa', 'Species', 'Category', 'Pathology', 'Epitope.peptide', 'MHC')]

In [10]:
dim(dat)

[1] 33559     6

In [11]:
head(dat)

,CDR3.beta.aa,Species,Category,Pathology,Epitope.peptide,MHC
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
75,AAGETSGVSYNEQ,Human,Cancer,Melanoma,NA,HLA-A*02
76,ASRPTITVPYSNQPQH,Human,Cancer,Melanoma,NA,HLA-A*01
77,ASRPTITVPYSNQPQH,Human,Cancer,Melanoma,NA,HLA-A*01
78,ASSLVVWDRGGNOPQH,Human,Cancer,Melanoma,NA,HLA-A*02
79,ASSQDLLSWDEQ,Human,Cancer,Melanoma,NA,HLA-A*02
93,ASSLGNEQ,Human,Cancer,Melanoma,EAAGIGILTV,HLA-A*02


In [12]:
# remove unclear epitope
dat <- unique(dat[!is.na(dat$Epitope.peptide) & !grepl("[[:punct:]]", dat$Epitope.peptide),])
dim(dat)

[1] 10415     6

In [13]:
head(dat)

,CDR3.beta.aa,Species,Category,Pathology,Epitope.peptide,MHC
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
93,ASSLGNEQ,Human,Cancer,Melanoma,EAAGIGILTV,HLA-A*02
94,ASSLGVATGEL,Human,Cancer,Melanoma,EAAGIGILTV,HLA-A*02
95,ASSQEEGGGSWGNTIY,Human,Cancer,Melanoma,EAAGIGILTV,HLA-A*02
96,ASSQEGLAGASQY,Human,Cancer,Melanoma,EAAGIGILTV,HLA-A*02
97,ASSQETDIVFNOPQH,Human,Cancer,Melanoma,EAAGIGILTV,HLA-A*02
98,ASSQGQLTDTQY,Human,Cancer,Melanoma,EAAGIGILTV,HLA-A*02


In [14]:
# write loaded data
write.table(dat, 'McPAS-loaded.csv', quote = F, row.names = F, sep = ',', append = F)

# Analysis

In [15]:
unique(dat$Pathology)

[1] "Melanoma"                           "Epstein Barr virus (EBV)"          
 [3] "Multiple sclerosis (MS)"            "Cytomegalovirus (CMV)"             
 [5] "Diabetes Type 1"                    "Human immunodeficiency virus (HIV)"
 [7] "Herpes simplex virus 2 (HSV2)"      "Hepatitis C virus"                 
 [9] "Allergy"                            "Influenza"                         
[11] "Acute myeloid leukemia"             "Lung cancer"                       
[13] "Celiac disease"                     "Yellow fever virus"                
[15] "M. tuberculosis"                    "Neoantigen"                        
[17] "Hepatitis C virus (HCV)"            "Diabetes type 2"                   
[19] "Lymphoma"                           "Human herpes virus 1"              
[21] "Human papilloma virus"              "HTLV-1"                            
[23] "pollen allergen"                    "Merkel cell carcinoma"             
[25] "Epithelial ovarian cancer"          "Alzheimer's disease"               
[27] "HTLV-1 (Chronic"                    "Hepatitis E virus infection (cHEV)"
[29] NA                                   "Tumor associated antigen (TAA)"

In [16]:
unique(dat$Category)

[1] "Cancer"     "Pathogens"  "Autoimmune" "Allergy"

In [17]:
head(dat)

,CDR3.beta.aa,Species,Category,Pathology,Epitope.peptide,MHC
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
93,ASSLGNEQ,Human,Cancer,Melanoma,EAAGIGILTV,HLA-A*02
94,ASSLGVATGEL,Human,Cancer,Melanoma,EAAGIGILTV,HLA-A*02
95,ASSQEEGGGSWGNTIY,Human,Cancer,Melanoma,EAAGIGILTV,HLA-A*02
96,ASSQEGLAGASQY,Human,Cancer,Melanoma,EAAGIGILTV,HLA-A*02
97,ASSQETDIVFNOPQH,Human,Cancer,Melanoma,EAAGIGILTV,HLA-A*02
98,ASSQGQLTDTQY,Human,Cancer,Melanoma,EAAGIGILTV,HLA-A*02


In [18]:
dat <- unique(dat[, c('CDR3.beta.aa', 'MHC', 'Epitope.peptide', 'Pathology')])
dim(dat)

[1] 10415     4

In [19]:
length(unique(dat$MHC))
sort(unique(dat$MHC))

[1] 62

[1] "DQ8-trans"      "DR3*02:02"      "DRB1*04-01"     "DRB1*04:01"    
 [5] "DRB1*15:03"     "H-2Kb"          "HLA-A*01"       "HLA-A*011"     
 [9] "HLA-A*01:01"    "HLA-A*02"       "HLA-A*02:01"    "HLA-A*02:02"   
[13] "HLA-A*02:03"    "HLA-A*02:04"    "HLA-A*02:05"    "HLA-A*02:06"   
[17] "HLA-A*02:07"    "HLA-A*02:08"    "HLA-A*02:09"    "HLA-A*02:10"   
[21] "HLA-A*02:11"    "HLA-A*02:12"    "HLA-A*02:13"    "HLA-A*02:14"   
[25] "HLA-A*02:15"    "HLA-A*02:16"    "HLA-A*02:17"    "HLA-A*11:01"   
[29] "HLA-A*24:02"    "HLA-A*2:01"     "HLA-A1"         "HLA-A2"        
[33] "HLA-A2:01"      "HLA-B*07:02"    "HLA-B*08"       "HLA-B*08:01"   
[37] "HLA-B*15"       "HLA-B*27"       "HLA-B*27:05"    "HLA-B*35:01"   
[41] "HLA-B*35:02"    "HLA-B*42"       "HLA-B*42:01"    "HLA-B*44:05"   
[45] "HLA-B*57"       "HLA-B*57:01"    "HLA-B*57:03"    "HLA-B*8"       
[49] "HLA-B7"         "HLA-C*07:02"    "HLA-Cw* 16:01"  "HLA-DPB1*02:01"
[53] "HLA-DQ2"        "HLA-DQ2.5"      "HLA-DQ8"        "HLA-DR1"       
[57] "HLA-DR11"       "HLA-DR15"       "HLA-DR5"        "HLA-DRB1"      
[61] "HLA-DRB1*04:01"

In [20]:
sort(table(dat$MHC))


     DQ8-trans          H-2Kb    HLA-A*02:02    HLA-A*02:03    HLA-A*02:04 
             1              1              1              1              1 
   HLA-A*02:05    HLA-A*02:06    HLA-A*02:07    HLA-A*02:08    HLA-A*02:09 
             1              1              1              1              1 
   HLA-A*02:10    HLA-A*02:11    HLA-A*02:12    HLA-A*02:16    HLA-A*02:17 
             1              1              1              1              1 
      HLA-B*42       HLA-A*01    HLA-A*02:13    HLA-A*02:14    HLA-A*02:15 
             1              2              2              2              2 
   HLA-B*35:02    HLA-B*44:05      DR3*02:02        HLA-B*8  HLA-Cw* 16:01 
             2              2              3              3              4 
      HLA-B*15     DRB1*04:01     DRB1*15:03     HLA-A*2:01      HLA-DQ2.5 
             5              7              7              7              7 
       HLA-DQ8 HLA-DRB1*04:01 HLA-DPB1*02:01       HLA-DRB1      HLA-A*011 
           

In [21]:
# In CDR3-epitope prediction, HLA alleles are not needed. Therefore, unclear epitopes are kept.
dat_wohla <- unique(dat[, c('CDR3.beta.aa', 'Epitope.peptide')])
dim(dat_wohla)

[1] 10232     2

In [22]:
head(dat_wohla)
write.table(dat_wohla, 'McPAS-without-HLA.csv', quote = F, row.names = F, append = F, sep = ',')

,CDR3.beta.aa,Epitope.peptide
,<chr>,<chr>
93,ASSLGNEQ,EAAGIGILTV
94,ASSLGVATGEL,EAAGIGILTV
95,ASSQEEGGGSWGNTIY,EAAGIGILTV
96,ASSQEGLAGASQY,EAAGIGILTV
97,ASSQETDIVFNOPQH,EAAGIGILTV
98,ASSQGQLTDTQY,EAAGIGILTV


In [23]:
dat <- dat[!is.na(dat$MHC) & dat$MHC != '-' & !grepl('[/]', dat$MHC),]
table(dat$MHC)


     DQ8-trans      DR3*02:02     DRB1*04-01     DRB1*04:01     DRB1*15:03 
             1              3            364              7              7 
         H-2Kb       HLA-A*01      HLA-A*011    HLA-A*01:01       HLA-A*02 
             1              2             15             52            391 
   HLA-A*02:01    HLA-A*02:02    HLA-A*02:03    HLA-A*02:04    HLA-A*02:05 
          1284              1              1              1              1 
   HLA-A*02:06    HLA-A*02:07    HLA-A*02:08    HLA-A*02:09    HLA-A*02:10 
             1              1              1              1              1 
   HLA-A*02:11    HLA-A*02:12    HLA-A*02:13    HLA-A*02:14    HLA-A*02:15 
             1              1              2              2              2 
   HLA-A*02:16    HLA-A*02:17    HLA-A*11:01    HLA-A*24:02     HLA-A*2:01 
             1              1             15            203              7 
        HLA-A1         HLA-A2      HLA-A2:01    HLA-B*07:02       HLA-B*08 
           

In [24]:
dat$MHC <- gsub('HLA[-]', '', dat$MHC)
dat$MHC <- paste0('HLA-', dat$MHC)

In [25]:
table(dat$MHC)


      HLA-A*01      HLA-A*011    HLA-A*01:01       HLA-A*02    HLA-A*02:01 
             2             15             52            391           1284 
   HLA-A*02:02    HLA-A*02:03    HLA-A*02:04    HLA-A*02:05    HLA-A*02:06 
             1              1              1              1              1 
   HLA-A*02:07    HLA-A*02:08    HLA-A*02:09    HLA-A*02:10    HLA-A*02:11 
             1              1              1              1              1 
   HLA-A*02:12    HLA-A*02:13    HLA-A*02:14    HLA-A*02:15    HLA-A*02:16 
             1              2              2              2              1 
   HLA-A*02:17    HLA-A*11:01    HLA-A*24:02     HLA-A*2:01         HLA-A1 
             1             15            203              7            297 
        HLA-A2      HLA-A2:01    HLA-B*07:02       HLA-B*08    HLA-B*08:01 
          2756            191            466             46            208 
      HLA-B*15       HLA-B*27    HLA-B*27:05    HLA-B*35:01    HLA-B*35:02 
           

In [32]:
dat <- unique(dat[, c('CDR3.beta.aa', 'MHC', 'Epitope.peptide')])
dim(dat)

[1] 10335     3

In [33]:
write.table(dat, 'McPAS-analyzed.csv', quote = F, append = F, row.names = F, sep = ',')